# Homework 2
B06201053 數學四 鄭心慈

#### For plain vanilla calls and puts, implement the following option pricing methods:

## Basic requirement (80 points)

$
\left\{
    \begin{array}{ll}
        \mbox{Black-Scholes formulas (for European options)}\\
        \\
        \mbox{Monte Carlo simulation (for European options)}\\
        \\
        \mbox{CRR binomial tree model (for both European and American options)}
    \end{array}
\right.
$

>Inputs: $S_0$, $K$, $r$, $q$, $\sigma$, $T$, number of simulations, number of repetitions, $n$.

>Outputs: Option values for all methods and 95% confidence interval for Monte Carlo simulation.

In [ ]:
import numpy as np

In [ ]:
### input

In [ ]:
S_0=float(input('Please input S_0: '))

In [ ]:
K=float(input('Please input K: '))

In [ ]:
r=float(input('Please input r: '))

In [ ]:
q=float(input('Please input q: '))

In [ ]:
sigma=float(input('Please input sigma: '))

In [ ]:
T=float(input('Please input T: '))

In [ ]:
n_si=int(input('Please input number of simulations: '))

In [ ]:
n_re=int(input('Please input number of repetitions: '))

In [ ]:
n=int(input('Please input n: '))

In [ ]:
### Black-Scholes formulas

In [ ]:
from scipy.stats import norm

In [ ]:
### European call option
def BS_call_price(S_0, K, r, q, sigma, T):
    d_1=(np.log(S_0/K)+(r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))
    N_d_1=norm.cdf(d_1)
    d_2=(np.log(S_0/K)+(r-q-sigma**2/2)*T)/(sigma*np.sqrt(T))
    N_d_2=norm.cdf(d_2)
    price=S_0*np.exp(-q*T)*N_d_1-np.exp(-r*T)*K*N_d_2
    return price

print(BS_call_price(S_0, K, r, q, sigma, T))

In [ ]:
### European put option
def BS_put_price(S_0, K, r, q, sigma, T):
    d_1=(np.log(S_0/K)+(r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))
    m_N_d_1=norm.cdf(-d_1)
    d_2=(np.log(S_0/K)+(r-q-sigma**2/2)*T)/(sigma*np.sqrt(T))
    m_N_d_2=norm.cdf(-d_2)
    price=np.exp(-r*T)*K*m_N_d_2-S_0*np.exp(-q*T)*m_N_d_1
    return price

print(BS_put_price(S_0, K, r, q, sigma, T))

In [ ]:
### Monte Carlo simulation

In [ ]:
### European call option
def MC_call_price(S_0, K, r, q, sigma, T, n_si):
    S_T=np.exp(np.random.normal(np.log(S_0)+(r-q-sigma**2/2)*T, np.sqrt(sigma**2*T), n_si))
    payoff=np.zeros(n_si)
    for i in range(0, n_si, 1):
        if K<=S_T[i]:
            payoff[i]=S_T[i]-K
    price=np.exp(-r*T)*np.mean(payoff)
    return price

sample=np.zeros(n_re)
for i in range(0, n_re, 1):
    sample[i]=MC_call_price(S_0, K, r, q, sigma, T, n_si)

print('[', np.mean(sample)-2*np.std(sample), ',', np.mean(sample)+2*np.std(sample), ']')

In [ ]:
### European put option
def MC_put_price(S_0, K, r, q, sigma, T, n_si):
    S_T=np.exp(np.random.normal(np.log(S_0)+(r-q-sigma**2/2)*T, np.sqrt(sigma**2*T), n_si))
    payoff=np.zeros(n_si)
    for i in range(0, n_si, 1):
        if K>=S_T[i]:
            payoff[i]=K-S_T[i]
    price=np.exp(-r*T)*np.mean(payoff)
    return price

sample=np.zeros(n_re)
for i in range(0, n_re, 1):
    sample[i]=MC_put_price(S_0, K, r, q, sigma, T, n_si)

print('[', np.mean(sample)-2*np.std(sample), ',', np.mean(sample)+2*np.std(sample), ']')

In [ ]:
### CRR binomial tree model

In [ ]:
dt=T/n
u=np.exp(sigma*np.sqrt(dt))
d=1/u
p=(np.exp((r-q)*dt)-d)/(u-d)

In [ ]:
### European call option
def CRR_E_call_price(S_0, K, r, q, sigma, dt, n):
    S_t=np.zeros((n+1, n+1))
    for i in range(0, n+1, 1):
        for j in range(0, i+1, 1):
            S_t[i,j]=S_0*u**(i-j)*d**j
    c_t=np.zeros((n+1, n+1))
    for j in range(0, n+1, 1):
        c_t[n,j]=max(S_t[n,j]-K,0)
    for i in range(n-1, -1, -1):
        for j in range(0, i+1, 1):
            c_t[i,j]=np.exp(-r*dt)*(p*c_t[i+1,j]+(1-p)*c_t[i+1,j+1])
    return c_t[0,0]

print(CRR_E_call_price(S_0, K, r, q, sigma, dt, n))

In [ ]:
### European put option
def CRR_E_put_price(S_0, K, r, q, sigma, dt, n):
    S_t=np.zeros((n+1, n+1))
    for i in range(0, n+1, 1):
        for j in range(0, i+1, 1):
            S_t[i,j]=S_0*u**(i-j)*d**j
    p_t=np.zeros((n+1, n+1))
    for j in range(0, n+1, 1):
        p_t[n,j]=max(K-S_t[n,j],0)
    for i in range(n-1, -1, -1):
        for j in range(0, i+1, 1):
            p_t[i,j]=np.exp(-r*dt)*(p*p_t[i+1,j]+(1-p)*p_t[i+1,j+1])
    return p_t[0,0]

print(CRR_E_put_price(S_0, K, r, q, sigma, dt, n))

In [ ]:
### American call option
def CRR_A_call_price(S_0, K, r, q, sigma, dt, n):
    S_t=np.zeros((n+1, n+1))
    for i in range(0, n+1, 1):
        for j in range(0, i+1, 1):
            S_t[i,j]=S_0*u**(i-j)*d**j
    c_t=np.zeros((n+1, n+1))
    for j in range(0, n+1, 1):
        c_t[n,j]=max(S_t[n,j]-K,0)
        if K<=S_t[n,j]:
            c_t[n,j]=S_t[n,j]-K
    for i in range(n-1, -1, -1):
        for j in range(0, i+1, 1):
            c_t[i,j]=max(np.exp(-r*dt)*(p*c_t[i+1,j]+(1-p)*c_t[i+1,j+1]),S_t[i,j]-K)
    return c_t[0,0]

print(CRR_A_call_price(S_0, K, r, q, sigma, dt, n))

In [ ]:
### American put option
def CRR_A_put_price(S_0, K, r, q, sigma, dt, n):
    S_t=np.zeros((n+1, n+1))
    for i in range(0, n+1, 1):
        for j in range(0, i+1, 1):
            S_t[i,j]=S_0*u**(i-j)*d**j
    p_t=np.zeros((n+1, n+1))
    for j in range(0, n+1, 1):
        if K>=S_t[n,j]:
            p_t[n,j]=K-S_t[n,j]
    for i in range(n-1, -1, -1):
        for j in range(0, i+1, 1):
            p_t[i,j]=max(np.exp(-r*dt)*(p*p_t[i+1,j]+(1-p)*p_t[i+1,j+1]),K-S_t[i,j])
    return p_t[0,0]

print(CRR_A_put_price(S_0, K, r, q, sigma, dt, n))

## Bonus 1 (5 points)

Implement the CRR binomial tree with one column vector.

In [ ]:
dt=T/n
u=np.exp(sigma*np.sqrt(dt))
d=1/u
p=(np.exp((r-q)*dt)-d)/(u-d)

In [ ]:
### European call option
def CRR_E_call_price_1(S_0, K, r, q, sigma, dt, n):
    c_t=np.zeros(n+1)
    for j in range(0, n+1, 1):
        c_t[j]=max(S_0*u**(n-j)*d**j-K,0)
    for i in range(n-1, -1, -1):
        for j in range(0, i+1, 1):
            c_t[j]=np.exp(-r*dt)*(p*c_t[j]+(1-p)*c_t[j+1])
    return c_t[0]

print(CRR_E_call_price_1(S_0, K, r, q, sigma, dt, n))

In [ ]:
### European put option
def CRR_E_put_price_1(S_0, K, r, q, sigma, dt, n):
    p_t=np.zeros(n+1)
    for j in range(0, n+1, 1):
        p_t[j]=max(K-S_0*u**(n-j)*d**j,0)
    for i in range(n-1, -1, -1):
        for j in range(0, i+1, 1):
            p_t[j]=np.exp(-r*dt)*(p*p_t[j]+(1-p)*p_t[j+1])
    return p_t[0]

print(CRR_E_put_price_1(S_0, K, r, q, sigma, dt, n))

In [ ]:
### American call option
def CRR_A_call_price_1(S_0, K, r, q, sigma, dt, n):
    c_t=np.zeros(n+1)
    for j in range(0, n+1, 1):
        c_t[j]=max(S_0*u**(n-j)*d**j-K,0)
    for i in range(n-1, -1, -1):
        for j in range(0, i+1, 1):
            c_t[j]=max(np.exp(-r*dt)*(p*c_t[j]+(1-p)*c_t[j+1]),S_0*u**(i-j)*d**j-K)
    return c_t[0]

print(CRR_A_call_price_1(S_0, K, r, q, sigma, dt, n))

In [ ]:
### American put option
def CRR_A_put_price_1(S_0, K, r, q, sigma, dt, n):
    p_t=np.zeros(n+1)
    for j in range(0, n+1, 1):
        p_t[j]=max(K-S_0*u**(n-j)*d**j,0)
    for i in range(n-1, -1, -1):
        for j in range(0, i+1, 1):
            p_t[j]=max(np.exp(-r*dt)*(p*p_t[j]+(1-p)*p_t[j+1]),K-S_0*u**(i-j)*d**j)
    return p_t[0]

print(CRR_A_put_price_1(S_0, K, r, q, sigma, dt, n))

## Bonus 2 (5 points)

Implement the combinatorial method to price European options.

In [ ]:
dt=T/n
u=np.exp(sigma*np.sqrt(dt))
d=1/u
p=(np.exp((r-q)*dt)-d)/(u-d)

In [ ]:
### European call option
def CB_call_price(S_0, K, r, q, sigma, T, n):
    summ=np.zeros(n+1)
    for i in range(0, n+1, 1):
        if S_0*u**(n-i)*d**i>K:
            for j in range(1, i+1, 1):
                summ[i]-=np.log(j)
            for j in range(n-i+1, n+1, 1):
                summ[i]+=np.log(j)
            summ[i]+=(n-i)*np.log(p)+i*np.log(1-p)+np.log(S_0*u**(n-i)*d**i-K)
            summ[i]=np.exp(summ[i])
    return np.exp(-r*T)*sum(summ)

print(CB_call_price(S_0, K, r, q, sigma, T, n))

In [ ]:
### European put option
def CB_put_price(S_0, K, r, q, sigma, T, n):
    summ=np.zeros(n+1)
    for i in range(0, n+1, 1):
        if S_0*u**(n-i)*d**i<K:
            for j in range(1, i+1, 1):
                summ[i]-=np.log(j)
            for j in range(n-i+1, n+1, 1):
                summ[i]+=np.log(j)
            summ[i]+=(n-i)*np.log(p)+i*np.log(1-p)+np.log(K-S_0*u**(n-i)*d**i)
            summ[i]=np.exp(summ[i])
    return np.exp(-r*T)*sum(summ)

print(CB_put_price(S_0, K, r, q, sigma, T, n))